In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
avocado = pd.read_csv("dataset/avocado_2015_to_2020.csv")

In [7]:
avocado.head()

,AveragePrice,Total Volume,Small/Medium Hass,Large Hass,Extra Large Hass,Total Bags,Small Bags,Large Bags,XLarge Bags,Type,Year,Region,Month
0,1.75,27365.89,9307.34,3844.81,615.28,13598.46,13061.10,537.36,0.0,Organic,2015,Southeast,Jan
1,1.49,17723.17,1189.35,15628.27,0.00,905.55,905.55,0.00,0.0,Organic,2015,Chicago,Jan
2,1.68,2896.72,161.68,206.96,0.00,2528.08,2528.08,0.00,0.0,Organic,2015,HarrisburgScranton,Jan
3,1.52,54956.80,3013.04,35456.88,1561.70,14925.18,11264.80,3660.38,0.0,Conventional,2015,Pittsburgh,Jan
4,1.64,1505.12,1.27,1129.50,0.00,374.35,186.67,187.68,0.0,Organic,2015,Boise,Jan


In [5]:
avocado = avocado.rename(columns={"4046": "Small/Medium Hass", "4225": "Large Hass", "4770": "Extra Large Hass"})

In [31]:
avocado.head()

,AveragePrice,Total Volume,Small/Medium Hass,Large Hass,Extra Large Hass,Total Bags,Small Bags,Large Bags,XLarge Bags,Type,Year,Region,Month
0,1.75,27365.89,9307.34,3844.81,615.28,13598.46,13061.10,537.36,0.0,Organic,2015,Southeast,Jan
1,1.49,17723.17,1189.35,15628.27,0.00,905.55,905.55,0.00,0.0,Organic,2015,Chicago,Jan
2,1.68,2896.72,161.68,206.96,0.00,2528.08,2528.08,0.00,0.0,Organic,2015,HarrisburgScranton,Jan
3,1.52,54956.80,3013.04,35456.88,1561.70,14925.18,11264.80,3660.38,0.0,Conventional,2015,Pittsburgh,Jan
4,1.64,1505.12,1.27,1129.50,0.00,374.35,186.67,187.68,0.0,Organic,2015,Boise,Jan


In [29]:
month_order = avocado["Month"].unique()

In [30]:
avocado.groupby("Month").agg({"AveragePrice": "mean", "Total Volume": "sum"}).reindex(month_order, axis= 0)

,AveragePrice,Total Volume
Month,,
Jan,1.287596,2.873170e+09
Feb,1.248885,2.839209e+09
Mar,1.316207,2.903505e+09
Apr,1.374333,2.711321e+09
May,1.367424,2.489171e+09
Jun,1.417295,2.249134e+09
Jul,1.480435,2.287320e+09
Aug,1.512892,1.972065e+09
Sep,1.525404,2.010691e+09
